# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
from scipy.stats import norm
import sys
import pandas as pd
import pickle

## Helper Functions

In [2]:
%store -r analysis_hf
sys.path.insert(0, analysis_hf)

import covariance as c
import visualization as viz

# Set Directory Paths

In [3]:
# Directory path to load formatted Pathology Data
%store -r path_dataDir

# Directory path to save Calcualted Analysis Data
%store -r path_CalData

# [1] Load Formatted Pathology Data (TAU and TDP)

### Loading GM Pathology Data

In [4]:
pathT_GM = pd.read_csv(os.path.join(path_dataDir, 'new_pathT(GM).csv'))

In [5]:
pathT_GM

,INDDID,FullAutopsyID,AutopsyIDNumOnly,Tau1_TDP2,AnalysisRegion,ANG_L,ATC_L,HIP_L,IFC_L,M1_L,...,SPC_R,V1_R,aCING_R,aINS_R,aITC_R,dlPFC_R,iPFC_R,mPFC_R,pCING_R,pSTC_R
0,100071.0,2015-075,2015-075,2.0,GM,0.005646,NaN,NaN,NaN,0.088999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100551.0,2016-146,2016-146,1.0,GM,5.185200,NaN,9.013840,28.979094,10.032761,...,5.533633,NaN,NaN,NaN,NaN,21.341771,16.325921,NaN,NaN,NaN
2,100686.0,2009-097,2009-097,2.0,GM,0.020508,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101068.0,2004-267 UCSF,2004-267,1.0,GM,0.259120,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101105.0,2002-159,2002-159,1.0,GM,0.243036,NaN,0.729886,NaN,2.039751,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,118765.0,2014-092,2014-092,2.0,GM,NaN,NaN,NaN,NaN,NaN,...,NaN,0.002238,0.001168,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,118935.0,2014-232,2014-232,2.0,GM,NaN,NaN,NaN,NaN,NaN,...,NaN,0.002314,0.076726,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,119010.0,2014-256,2014-256,2.0,GM,NaN,NaN,NaN,NaN,NaN,...,NaN,0.018718,0.000488,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,119052.0,2014-200,2014-200,2.0,GM,NaN,NaN,NaN,NaN,NaN,...,NaN,0.002337,0.001032,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Loading TAU Pathology Data

In [6]:
# path_TAU
with open(os.path.join(path_dataDir, 'path_TAU.pkl'), 'rb') as f:
    path_TAU = pickle.load(f)
f.close()

In [7]:
print(path_TAU.shape)

(76, 40)


### Loading TDP Pathology Data

In [8]:
# path_TDP
with open(os.path.join(path_dataDir, 'path_TDP.pkl'), 'rb') as f:
    path_TDP = pickle.load(f)
f.close()

In [9]:
print(path_TDP.shape)

(103, 40)


# [2] Parameter/Variable Setting

### P-value Threshold List

In [10]:
# pthresh_list = [0.05]
pthresh = 0.05

### Covariance Matrix Threshold (to remove noise)

In [11]:
cov_thresh = 0.1
# cov_thresh = -10.0

### Label (Pathology Region) Names we are able to map to 3D. In alphabetical order (_L first and then _R)

In [12]:
LabelNames = pathT_GM.columns.values[5:]

In [13]:
len(LabelNames)

40

In [14]:
LabelNames

array(['ANG_L', 'ATC_L', 'HIP_L', 'IFC_L', 'M1_L', 'MFC_L', 'OFC_L',
       'PC_L', 'S1_L', 'SMTC_L', 'SPC_L', 'V1_L', 'aCING_L', 'aINS_L',
       'aITC_L', 'dlPFC_L', 'iPFC_L', 'mPFC_L', 'pCING_L', 'pSTC_L',
       'ANG_R', 'ATC_R', 'HIP_R', 'IFC_R', 'M1_R', 'MFC_R', 'OFC_R',
       'PC_R', 'S1_R', 'SMTC_R', 'SPC_R', 'V1_R', 'aCING_R', 'aINS_R',
       'aITC_R', 'dlPFC_R', 'iPFC_R', 'mPFC_R', 'pCING_R', 'pSTC_R'],
      dtype=object)

# [3] Calculate Covariance Matrices

## 1. TAU

In [15]:
path_TAU.shape

(76, 40)

In [16]:
path_TDP.shape

(103, 40)

In [17]:
covTAU = c.covCal(path_TAU, path_TAU, cov_thresh)

## 2. TDP

In [18]:
covTDP = c.covCal(path_TDP, path_TDP, cov_thresh)

## 3. TAU > TDP and TDP > TAU

In [19]:
covTAU_gt_TDP, covTDP_gt_TAU = c.covCalSigXY(path_TAU, path_TDP, covTAU, covTDP, pthresh, cov_thresh)

## 4. TAU > TDP and TDP > TAU RAW

In [20]:
covTAU_gt_TDP_raw, covTDP_gt_TAU_raw = c.covCalSigXYRaw(path_TAU, path_TDP, covTAU, covTDP, cov_thresh)

# Saving

### Save the Covariance Matrices

In [21]:
# covTAU
with open(os.path.join(path_CalData, 'covTAU.pkl'), 'wb') as f:
    pickle.dump(covTAU, f)
f.close()

# covTDP
with open(os.path.join(path_CalData, 'covTDP.pkl'), 'wb') as f:
    pickle.dump(covTDP, f)
f.close()

# covTAU_gt_TDP
with open(os.path.join(path_CalData, 'covTAU_gt_TDP.pkl'), 'wb') as f:
    pickle.dump(covTAU_gt_TDP, f)
f.close()

# covTDP_gt_TAU
with open(os.path.join(path_CalData, 'covTDP_gt_TAU.pkl'), 'wb') as f:
    pickle.dump(covTDP_gt_TAU, f)
f.close()

# covTAU_gt_TDP_raw
with open(os.path.join(path_CalData, 'covTAU_gt_TDP_raw.pkl'), 'wb') as f:
    pickle.dump(covTAU_gt_TDP_raw, f)
f.close()

# covTDP_gt_TAU_raw
with open(os.path.join(path_CalData, 'covTDP_gt_TAU_raw.pkl'), 'wb') as f:
    pickle.dump(covTDP_gt_TAU_raw, f)
f.close()

### Save LabelNames

In [22]:
# LabelNames
with open(os.path.join(path_CalData, 'LabelNames.pkl'), 'wb') as f:
    pickle.dump(LabelNames, f)
f.close()

In [23]:
pd.DataFrame(covTAU)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,NaN,NaN,0.476448,0.865188,0.376753,0.820083,0.845703,NaN,NaN,0.785595,...,NaN,NaN,0.323591,0.781064,0.849838,0.727331,0.683048,NaN,NaN,NaN
1,NaN,NaN,NaN,0.697745,NaN,0.344234,0.828246,NaN,NaN,0.803091,...,NaN,NaN,NaN,NaN,NaN,0.526617,0.833758,NaN,NaN,NaN
2,0.476448,NaN,NaN,0.707156,NaN,0.364779,0.563492,NaN,NaN,0.407876,...,NaN,NaN,0.849721,NaN,NaN,0.896187,0.866323,NaN,0.121724,NaN
3,0.865188,0.697745,0.707156,NaN,0.289373,0.844092,0.812043,NaN,NaN,0.937182,...,0.322477,NaN,0.694628,0.706885,0.834873,0.778356,0.885687,NaN,NaN,NaN
4,0.376753,NaN,NaN,0.289373,NaN,0.470357,0.317238,NaN,NaN,0.252127,...,NaN,NaN,0.146290,0.103608,NaN,NaN,NaN,NaN,NaN,NaN
5,0.820083,0.344234,0.364779,0.844092,0.470357,NaN,0.868088,NaN,NaN,0.789894,...,NaN,NaN,0.421689,0.781631,0.758508,0.640319,0.774824,NaN,NaN,NaN
6,0.845703,0.828246,0.563492,0.812043,0.317238,0.868088,NaN,NaN,NaN,0.786640,...,NaN,NaN,0.747908,0.870755,0.945220,0.573893,0.821498,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.785595,0.803091,0.407876,0.937182,0.252127,0.789894,0.786640,NaN,NaN,NaN,...,0.284836,NaN,0.220975,0.598061,0.892186,0.786588,0.867853,NaN,0.244553,NaN
